In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from skimage.metrics import structural_similarity as SSIM
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import PIL
from os import listdir
from PIL import Image
import random
import numpy.ma as ma
from skimage.metrics import structural_similarity as SSIM
from WGAIN_model import *
print(tf.__version__)
gpus = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))

2.8.2
Num GPUs Available:  0


In [3]:
img = tf.io.read_file('./images/cat.png')
img = tf.image.decode_png(img, channels=1)
img = tf.image.convert_image_dtype(img, tf.float32)
img = tf.image.resize(img, (117, 117))
img = ma.masked_greater(img, 0)
custom_img=img.mask.astype(int)
custom_img = 1 - custom_img

In [10]:
def decode_img(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img
def prepare_noise_eval_dataset(ds, batch_size, side, throw_ratio):
    def introduce_missingness(X):
        # per row ratios how much should be thrown (each row a different number, max is max_ratio_to_throw)
        ratio_to_throw = throw_ratio*tf.ones([X.shape[0], side, side, 1])
        # binary mask of which features are left (0 - missing, 1 - OK)
        mask = tf.dtypes.cast(tf.math.greater(tf.random.uniform([X.shape[0], side, side, 1]), ratio_to_throw), tf.float32)
        # Do the mask to have 3 channels
        mask3 = tf.repeat(mask, [3], axis = 3)
        # prepare randomness
        randZ = tf.random.normal([X.shape[0], side, side, 3], mean = 0.0, stddev = 0.1)
        randZ = tf.math.multiply(1 - mask3, randZ)
        # apply the mask - leave original and substitute randoms for missings
        newX = tf.math.multiply(mask3, X)
        # return a training pair
        return (X, newX, mask, randZ)
    # do the preparation
    ds = ds.map(decode_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.batch(batch_size,drop_remainder = True)
    ds = ds.map(introduce_missingness, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
    return ds
def prepare_singlesquare_eval_dataset(ds, batch_size, side):
    def introduce_missingness(X):
        # centered square
        sides = np.ones(shape=(X.shape[0],))*int(side/2.0)
        corner_l = int(side / 2.0) -  (sides/2.0).astype(int)
        corner_u = int(side / 2.0) +  (sides/2.0).astype(int)
        center_mask = np.ones((X.shape[0], side, side, 1)).astype('float32')
        for i in range(X.shape[0]):
            center_mask[i,corner_l[i]:corner_u[i],corner_l[i]:corner_u[i],:] = 0
        # binary mask of which features are left (0 - missing, 1 - OK)
        mask = tf.constant(center_mask, dtype=tf.float32)
        # Do the mask to have 3 channels
        mask3 = tf.repeat(mask, [3], axis = 3)
        # prepare randomness
        randZ = tf.random.normal([X.shape[0], side, side, 3], mean = 0.0, stddev = 0.1)
        randZ = tf.math.multiply(1 - mask3, randZ)
        # apply the mask - leave original and substitute randoms for missings
        newX = tf.math.multiply(mask3, X)
        # return a training pair
        return (X, newX, mask, randZ)
    # do the preparation
    ds = ds.map(decode_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.batch(batch_size,drop_remainder = True)
    ds = ds.map(introduce_missingness, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
    return ds

def prepare_multisquare_eval_dataset(ds, batch_size, side):
    def introduce_missingness(X):
        # multi square
        DAMAGE = 0.3
        SQUARES = 5.0
        image_area = side*side
        damage_area = DAMAGE*image_area
        one_square_area = damage_area / SQUARES
        one_square_side = int(np.sqrt(one_square_area))
        test_mask = np.ones((X.shape[0],side,side,1))
        for i in range(test_mask.shape[0]):
            for k in range(int(SQUARES)):
                x = np.random.randint(side - one_square_side)
                y = np.random.randint(side - one_square_side)
                test_mask[i,x:x+one_square_side,y:y+one_square_side,0] = 0
        # binary mask of which features are left (0 - missing, 1 - OK)
        mask = tf.constant(test_mask, dtype=tf.float32)
        # Do the mask to have 3 channels
        mask3 = tf.repeat(mask, [3], axis = 3)
        # prepare randomness
        randZ = tf.random.normal([X.shape[0], side, side, 3], mean = 0.0, stddev = 0.1)
        randZ = tf.math.multiply(1 - mask3, randZ)
        # apply the mask - leave original and substitute randoms for missings
        newX = tf.math.multiply(mask3, X)
        # return a training pair
        return (X, newX, mask, randZ)
    # do the preparation
    ds = ds.map(decode_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.batch(batch_size,drop_remainder = True)
    ds = ds.map(introduce_missingness, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
    return ds
def prepare_customMask_eval_dataset(ds, batch_size, side):
    def decode_img(file_path):
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        return img
    def introduce_missingness(X):
        # Custom Mask
        corner_l = np.random.randint(low = 5, high = 10, size=batch_size)
        corner_u = int(side / 2.0)  +  corner_l + 53
        center_mask = np.ones((batch_size, side, side, 1)).astype('float32')
        for i in range(batch_size):
            center_mask[i,corner_l[i]:corner_u[i],corner_l[i]:corner_u[i],:] = custom_img
        mask = tf.constant(center_mask, dtype=tf.float32)
        
        mask3 = tf.repeat(mask, [3], axis = 3)
        # prepare randomness
        randZ = tf.random.normal([X.shape[0], side, side, 3], mean = 0.0, stddev = 0.1)
        randZ = tf.math.multiply(1 - mask3, randZ)
        # apply the mask - leave original and substitute randoms for missings
        newX = tf.math.multiply(mask3, X)
        # return a training pair
        return (X, newX, mask, randZ)
    # do the preparation
    ds = ds.map(decode_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    #ds = ds.shuffle(buffer_size = 1024)
    ds = ds.batch(batch_size,drop_remainder = True)
    ds = ds.map(introduce_missingness, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
    return ds
def evaluate_dataset(dataset, batch_size, side, print_log = True, max_batch = 5):
    count = 0
    psnr = 0
    ssim = 0
    for i, (origX, newX, mask, randZ) in enumerate(dataset):
        if max_batch and i > max_batch:
            break
        if print_log:
            print(i)
        imputed = generator([newX,mask,randZ])
        imputed = mask*origX + (1-mask)*imputed
        count += imputed.shape[0]
        for j in range(imputed.shape[0]):
            ssim += SSIM(origX[j,:,:,:].numpy(), imputed[j,:,:,:].numpy(), multichannel=True)
            psnr += tf.image.psnr(origX[j,:,:,:], imputed[j,:,:,:], 1)
        if print_log:
            print("sample count:", count)
            print("avg test PSNR: ", psnr/count)
            print("avg test SSIM: ", ssim/count)
    return count, psnr/count, ssim/count

In [11]:
batch_size = 32
side = 128

path = "./datasets/celeba/%s128"
celeba_test = data.Dataset.list_files(path%'test' + str('/*.jpg'))
celeba_test_d = prepare_multisquare_eval_dataset(celeba_test, batch_size, side)

# create the model
generator = build_generator(side)
# generator.summary()
critic = build_critic(side)
# critic.summary()

# Load weights
model_path = "saved_models_celeba_old/"

generator.load_weights(model_path + "gultimate_final")
critic.load_weights(model_path + "cultimate_final")

In [13]:
counts = []
psnrs = []
ssims = []
scenarios = ['center', 'multi', 'noise 50', 'noise 75', 'noise 95', 'customMask']
datasets = [
    prepare_singlesquare_eval_dataset(celeba_test, batch_size, side),
    prepare_multisquare_eval_dataset(celeba_test, batch_size, side),
    prepare_noise_eval_dataset(celeba_test, batch_size, side, 0.50),
    prepare_noise_eval_dataset(celeba_test, batch_size, side, 0.75),
    prepare_noise_eval_dataset(celeba_test, batch_size, side, 0.95),
    prepare_customMask_eval_dataset(celeba_test, batch_size, side)
]
for i in range(len(datasets)):
    print(scenarios[i])
    _count, _psnr, _ssim = evaluate_dataset(datasets[i], batch_size, side, False, max_batch=100)
    print(_count, _psnr, _ssim)
    counts.append(_count)
    psnrs.append(_psnr)
    ssims.append(_ssim)

center
96 tf.Tensor(26.315775, shape=(), dtype=float32) 0.9238268312184464
multi
96 tf.Tensor(27.293007, shape=(), dtype=float32) 0.9305722618792864
noise 50
96 tf.Tensor(34.03396, shape=(), dtype=float32) 0.9769218633283111
noise 75
96 tf.Tensor(30.140223, shape=(), dtype=float32) 0.9479118590056109
noise 95
96 tf.Tensor(23.983976, shape=(), dtype=float32) 0.8364108153944247
customMask
96 tf.Tensor(26.18847, shape=(), dtype=float32) 0.9187091383006473
